In [1]:
# ボツ
# 間違って動かしたら、escで終了できるよー

import cv2

# 読み込む動画の設定
cap = cv2.VideoCapture(r"C:\Users\kaneko-227\Videos\Captures\20230928.mp4")

while True:
    # フレームの読み込み
    ret, frame = cap.read()

    # 動画が終了したらループを抜ける
    if not ret:
        break

    # 結果画像の表示
    cv2.imshow("frame", frame)

    # ESCキーが押されたらループを抜ける
    if cv2.waitKey(30) & 0xff == 27:  # 27はESCキーのキーコード
        break

# キャプチャを解放し、ウィンドウを閉じる
cap.release()
cv2.destroyAllWindows()


In [1]:
# 1-2）AKAZE法で特徴検出するコード

import cv2
import numpy as np

# 動画のパス
video_path = r"C:\Users\kaneko-227\Videos\Captures\ネカ録 スーパーマルチビューア   [全カメラモード] 2023-12-06 11-46-11.mp4"

# Lucas-Kanade法のパラメータ
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# AKAZE検出器の作成
akaze = cv2.AKAZE_create()

# カラーで追跡するためのランダムな色を生成
color = np.random.randint(0, 255, (500, 3))

# ビデオをキャプチャ
cap = cv2.VideoCapture(video_path)

# 最初のフレームを読み込み、AKAZE法で特徴点を見つける
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
kp_akaze = akaze.detect(old_gray, None)
p0 = cv2.KeyPoint_convert(kp_akaze)

# マスク画像を作成して描画する
mask = np.zeros_like(old_frame)

# フレームレートを取得
fps = cap.get(cv2.CAP_PROP_FPS)

# 10秒間のフレーム数
frame_count = int(10 * fps)

for i in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # オプティカルフローを計算
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # 良い点を選択
    # st配列を適切に整形
    st = st.ravel() == 1

    good_new = p1[st]
    good_old = p0[st]

    # トラックを描画する際の色配列のサイズを動的に調整
    num_points = len(good_new)
    color = np.random.randint(0, 255, (num_points, 3))

    # トラックを描画
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)

    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # 現在のフレームとポイントを更新
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()
